In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# ✅ MONTAJE DE DRIVE Y CARGA DE MODELO + SCALERS
import joblib
import pandas as pd

# Ruta de tu carpeta
base_path = "/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/MIAA_taller02"

# Cargar modelo y scaler
model = joblib.load(f"{base_path}/model/modelo_xgboost_v1.pkl")
scaler_X = joblib.load(f"{base_path}/model/scaler_X.pkl")
scaler_y = joblib.load(f"{base_path}/model/scaler_y.pkl")

# Cargar dataset con datos futuros
df_cmip6 = pd.read_csv(f"{base_path}/data/df_cmip6.csv")

print("✅ Modelo y scaler cargados correctamente.")

✅ Modelo y scaler cargados correctamente.


## Función para visualizar el balance hídrico mensual suavizado con GEE y geemap

In [3]:
import ee
import geemap.foliumap as geemap

# Inicializar GEE (por si aún no lo hiciste)
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize(project="ee-freddyvillota")

# Definir la región de estudio (Provincia de Carchi)
roi = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1") \
    .filter(ee.Filter.eq("ADM0_NAME", "Ecuador")) \
    .filter(ee.Filter.eq("ADM1_NAME", "Carchi"))

# Función para calcular PET con Penman-Monteith (FAO-56)
def calc_pet(img):
    T = img.select('temperature_2m').subtract(273.15)
    Td = img.select('dewpoint_temperature_2m').subtract(273.15)
    Ra = img.select('surface_net_solar_radiation').add(img.select('surface_net_thermal_radiation')).divide(1e6)
    date = img.date()
    days = ee.Image.constant(date.advance(1, 'month').difference(date, 'day'))
    Rn = Ra.divide(days)
    G = ee.Image.constant(0)
    es = T.expression('0.6108 * exp(17.27 * T / (T + 237.3))', {'T': T})
    ea = Td.expression('0.6108 * exp(17.27 * Td / (Td + 237.3))', {'Td': Td})
    delta = es.multiply(4098).divide(T.add(237.3).pow(2))
    gamma = ee.Image.constant(1.013e-3 * 101.3 / (0.622 * 2.45))
    u = img.select('u_component_of_wind_10m')
    v = img.select('v_component_of_wind_10m')
    wind = u.pow(2).add(v.pow(2)).sqrt().multiply(4.87).divide(ee.Number(67.8 * 10 - 5.42).log())
    pet = delta.multiply(Rn.subtract(G)).multiply(0.408) \
        .add(gamma.multiply(900).divide(T.add(273)).multiply(wind).multiply(es.subtract(ea))) \
        .divide(delta.add(gamma.multiply(ee.Image.constant(1).add(wind.multiply(0.34)))))
    return pet.rename('PET')

# Función para mostrar el mapa
def mostrar_balance_hidrico(year, month):
    era5 = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY") \
        .filterDate(f"{year}-{month:02d}-01", f"{year}-{month:02d}-28")

    petCol = era5.map(calc_pet)
    precipCol = era5.select('total_precipitation') \
        .map(lambda img: img.multiply(1000).rename('precip_mm').copyProperties(img, img.propertyNames()))

    pet = petCol.first()
    pr = precipCol.first()
    balance = pr.subtract(pet).rename('balance_mm').clip(roi)
    balance_smooth = balance.resample('bicubic')

    stats = balance.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=roi.geometry(),
        scale=10000,
        bestEffort=True
    ).getInfo()

    vmin = stats['balance_mm_min']
    vmax = stats['balance_mm_max']

    vis = {
        'min': vmin,
        'max': vmax,
        'palette': ['red', 'yellow', 'lightgreen', 'green', 'blue']
    }

    Map = geemap.Map(center=[0.95, -78.4], zoom=8)
    Map.addLayer(balance_smooth, vis, f'Balance hídrico {year}-{month:02d}')
    Map.addLayer(roi.style(color='black', width=2, fillColor='00000000'), {}, 'Carchi')
    return Map.to_html()

In [4]:
# ✅ Flask app para visualización + predicción múltiple
from flask import Flask, request, render_template_string
from pyngrok import ngrok, conf
import threading
import matplotlib.pyplot as plt
import io
import base64

# Configurar ngrok
conf.get_default().auth_token = "2xIQTToHZNmpsRT8pufzVYzLWPl_6owaAbgx9TU4vCCU9zD2k"
ngrok.kill()

# Iniciar Flask
app = Flask(__name__)

@app.route("/")
def index():
    return '''
    <h2>🌧️ Balance hídrico mensual</h2>
    <form action="/balance">
      Año: <input type="number" name="year" value="2020"><br>
      Mes: <input type="number" name="month" value="1"><br>
      <input type="submit" value="Ver mapa">
    </form>
    <br><hr>
    <h2>📈 Predicción de balance futuro</h2>
    <form action="/predecir">
      Año inicial: <input type="number" name="year_ini" value="2030"><br>
      Año final: <input type="number" name="year_end" value="2050"><br>
      <input type="submit" value="Predecir">
    </form>
    '''

@app.route("/balance")
def balance():
    try:
        year = int(request.args.get("year"))
        month = int(request.args.get("month"))
        html_map = mostrar_balance_hidrico(year, month)
        return render_template_string(html_map)
    except Exception as e:
        return f"<p>Error generando el mapa: {e}</p>"

@app.route("/predecir")
def predecir():
    try:
        year_ini = int(request.args.get("year_ini"))
        year_end = int(request.args.get("year_end"))

        # Asegurar columna date
        df_cmip6['date'] = pd.to_datetime(df_cmip6['date'])

        # Filtrar por rango de fechas
        df_pred = df_cmip6[
            (df_cmip6['date'].dt.year >= year_ini) &
            (df_cmip6['date'].dt.year <= year_end)
        ]

        if df_pred.empty:
            return f"<p>❌ No hay datos entre {year_ini} y {year_end}</p>"

        # Escalar entradas y predecir
        X = scaler_X.transform(df_pred[['precip_mm', 'temp_c', 'wind_u', 'wind_v', 'solar_rad']])
        y_scaled = model.predict(X).reshape(-1, 1)
        y_real = scaler_y.inverse_transform(y_scaled).flatten()

        # Agregar al DataFrame
        df_pred['pred_balance'] = y_real

        # Crear gráfico tipo timeline
        import matplotlib.pyplot as plt
        import matplotlib.dates as mdates
        import io, base64

        fig, ax = plt.subplots(figsize=(12, 4))
        ax.plot(df_pred['date'], df_pred['pred_balance'], color='darkred', label='Predicción XGBoost')
        ax.set_title(f"Proyección del balance hídrico con XGBoost ({year_ini}–{year_end})")
        ax.set_xlabel("Año")
        ax.set_ylabel("Balance hídrico estimado (mm)")
        ax.legend()
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.xaxis.set_major_locator(mdates.YearLocator(5))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        fig.autofmt_xdate()

        buf = io.BytesIO()
        plt.tight_layout()
        plt.savefig(buf, format='png')
        buf.seek(0)
        img_base64 = base64.b64encode(buf.read()).decode('utf-8')
        buf.close()

        return f"""
        <h2>📊 Balance hídrico proyectado ({year_ini}–{year_end})</h2>
        <img src="data:image/png;base64,{img_base64}" style="max-width:100%;">
        """
    except Exception as e:
        return f"<p>❌ Error realizando la predicción: {e}</p>"

# Crear túnel con ngrok
public_url = ngrok.connect(5000)
print("🌐 Accede a tu app en:", public_url)

# Ejecutar Flask en segundo plano
def run():
    app.run(host="0.0.0.0", port=5000)

thread = threading.Thread(target=run)
thread.start()

🌐 Accede a tu app en: NgrokTunnel: "https://1ca9-35-229-66-103.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
